In [93]:
import pandas as pd
import os
import numpy as np

In [15]:
files = [
    r"C:/Users/PC/Documents/Water datasets/2019/2019 OTHERS.csv",
    r"C:/Users/PC/Documents/Water datasets/2019/2019 WA10 AND WA13.csv",
    r"C:/Users/PC/Documents/Water datasets/2019/2019 WA50.csv",
    r"C:/Users/PC/Documents/Water datasets/2019/2019 WA60-WA64.csv",
    r"C:/Users/PC/Documents/Water datasets/2019/2019 WA70.csv",
]

In [21]:
dfs = [pd.read_csv(f) for f in files]

In [39]:
combined2019 = pd.concat(dfs, ignore_index=True)

In [43]:
combined2019.to_excel(r"C:/Users/PC/Documents/Water datasets/2019/combined2019.xlsx", sheet_name="Combined2019", index=False)

In [45]:
# Check the exact column names
print(combined2019.columns)

Index(['ACCOUNT_NO', 'UNIT_NO', 'STAND_ADDRESS', 'TOWNSHIP', 'METER_NO',
       'METER_STATUS', 'TARIFF CODE', 'JANUARY', 'FEBRUARY', 'MARCH', 'APRIL',
       'MAY', 'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBR',
       'DECEMBER', 'Total', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22'],
      dtype='object')


In [51]:
# Drop any columns whose names start with 'Unnamed'
combined = combined.loc[:, ~combined.columns.str.contains('Unnamed')]

In [55]:
print(combined.columns)

Index(['ACCOUNT_NO', 'UNIT_NO', 'STAND_ADDRESS', 'TOWNSHIP', 'METER_NO',
       'METER_STATUS', 'TARIFF CODE', 'JANUARY', 'FEBRUARY', 'MARCH', 'APRIL',
       'MAY', 'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBR',
       'DECEMBER', 'Total'],
      dtype='object')


In [57]:
# Show all duplicated meter numbers (i.e., appearing more than once)
duplicates = combined[combined.duplicated(subset=['METER_NO'], keep=False)]

In [59]:
# Sort for easier viewing
duplicates = duplicates.sort_values(by='METER_NO')

In [61]:
print(f"Total duplicated meter numbers: {duplicates['METER_NO'].nunique()}")
duplicates.head(20)

Total duplicated meter numbers: 816


,ACCOUNT_NO,UNIT_NO,STAND_ADDRESS,TOWNSHIP,METER_NO,METER_STATUS,TARIFF CODE,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBR,DECEMBER,Total
9455,10616640,010000000076410000000000000,OMUKUNDA DRIVE,WINDHOEK/WINDHOEK BLOCKS ...,LZM652,ACTIVE,WA22,0,0,0,0,0,0,304,0,622,321,0,760,2007
54623,10616640,010000000076410000000000000,OMUKUNDA DRIVE,WINDHOEK/WINDHOEK BLOCKS ...,LZM652,ACTIVE,WA10,0,689,431,334,0,619,0,0,0,0,0,0,2073
783,10465561,180000000004940000000000000,HILDA ST,GOREANGAB ...,MZN418,ACTIVE,WA50,0,3481,2174,2165,1947,2404,0,3227,904,1007,1800,1601,20710
56337,10465561,180000000004940000000000000,HILDA ST,GOREANGAB ...,MZN418,ACTIVE,WA50,0,3481,2174,2165,1947,2404,0,3227,904,1007,1800,1601,20710
784,10749071,180000000004950000000000000,HILDA ST,GOREANGAB ...,NZN678,ACTIVE,WA50,0,3305,1588,1598,1513,1633,1534,1468,1917,868,1815,79,17318
56338,10749071,180000000004950000000000000,HILDA ST,GOREANGAB ...,NZN678,ACTIVE,WA50,0,3305,1588,1598,1513,1633,1534,1468,1917,868,1815,79,17318
1728,13273760,260000000028810000000000000,GOLGOTA ST,KATUTURA ...,BLAG326,ACTIVE,WA12,0,19,0,0,0,0,0,0,0,0,0,0,19
25630,13273760,260000000028810000000000000,GOLGOTA ST,KATUTURA ...,BLAG326,ACTIVE,WA10,0,0,16,16,10,14,37,27,0,13,13,8,154
25800,10406808,260000000026730000000000000,DIANA ST,KATUTURA ...,BLHH747,INACTIVE,WA10,0,0,0,0,0,0,2,1,0,2,1,0,6
1741,10406808,260000000026730000000000000,DIANA ST,KATUTURA ...,BLHH747,INACTIVE,WA12,0,1,1,1,1,0,0,0,0,0,0,0,4


In [63]:
months_12 = [
    'JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE',
    'JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBR','DECEMBER'
]

In [65]:
# Make sure columns exist and are numeric
months_12 = [c for c in months_12 if c in combined.columns]
combined[months_12] = combined[months_12].apply(pd.to_numeric, errors='coerce').fillna(0)

In [67]:
# Rule:
# 1️⃣ If METER_NO appears more than once with the SAME TARIFF CODE → keep first
# 2️⃣ If METER_NO appears with DIFFERENT TARIFF CODEs → sum month values
#     and keep the TARIFF CODE with the most non-zero months

In [69]:
# --- Step 1: find single- vs multi-tariff meters
tariff_counts = combined.groupby('METER_NO')['TARIFF CODE'].nunique()
single_meters = tariff_counts[tariff_counts == 1].index
multi_meters = tariff_counts[tariff_counts > 1].index

In [71]:
# --- Step 2: handle single-tariff meters
single_df = (combined[combined['METER_NO'].isin(single_meters)]
             .sort_values(['METER_NO'])
             .drop_duplicates(subset=['METER_NO'], keep='first'))

In [73]:
# --- Step 3: handle multi-tariff meters
multi_df = combined[combined['METER_NO'].isin(multi_meters)].copy()
multi_df['_nonzero_months'] = (multi_df[months_12].gt(0)).sum(axis=1)

In [75]:
# Count month entries per tariff
entries_per_tariff = (multi_df.groupby(['METER_NO','TARIFF CODE'])['_nonzero_months']
                      .sum().reset_index(name='month_entries'))

In [77]:
# Pick tariff with the most month entries
best_tariff = (entries_per_tariff.sort_values(['METER_NO','month_entries'], ascending=[True, False])
               .drop_duplicates(subset=['METER_NO']))

In [79]:
# Sum month totals across all rows per meter
summed = multi_df.groupby('METER_NO', as_index=False)[months_12].sum()

In [81]:
# Get one representative row from the chosen tariff per meter
rep_rows = (multi_df.merge(best_tariff[['METER_NO','TARIFF CODE']], on=['METER_NO','TARIFF CODE'])
                  .sort_values(['METER_NO'])
                  .groupby('METER_NO', as_index=False)
                  .first()[['METER_NO','ACCOUNT_NO','UNIT_NO','STAND_ADDRESS',
                            'TOWNSHIP','METER_STATUS','TARIFF CODE']])

In [83]:
# Merge summed months back in
multi_clean = rep_rows.merge(summed, on='METER_NO', how='left')
multi_clean['Total'] = multi_clean[months_12].sum(axis=1)

In [85]:
# --- Step 4: combine single + multi sets
cleaned = pd.concat([single_df, multi_clean], ignore_index=True)

In [87]:
# Recompute Total to be sure
cleaned['Total'] = cleaned[months_12].sum(axis=1)

In [89]:
# Optional flag
cleaned['MERGED_FROM_MULTI_TARIFF'] = cleaned['METER_NO'].isin(multi_meters).map({True:'Yes', False:'No'})

In [91]:
# --- Step 5: save output
output_path = r"C:/Users/PC/Documents/Water datasets/2019/combined2019_cleaned_by_rule.xlsx"
cleaned.to_excel(output_path, index=False)

In [95]:
months_12 = [
    'JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE',
    'JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBR','DECEMBER'
]

In [97]:
# ensure numeric months (zeros where not numeric)
cleaned[months_12] = cleaned[months_12].apply(pd.to_numeric, errors='coerce').fillna(0.0)

In [99]:
def backward_distribution_row(row, month_cols=months_12):
    vals = row[month_cols].astype(float).to_numpy().copy()

    # walk forward through months
    for i in range(len(vals)):
        if vals[i] > 0:
            # count consecutive zeros immediately before month i
            j = i - 1
            zeros = 0
            while j >= 0 and vals[j] == 0:
                zeros += 1
                j -= 1
            if zeros > 0:
                share = vals[i] / (zeros + 1)        # equal split over the block + this month
                for k in range(i - zeros, i + 1):    # fill the zero block and this month
                    vals[k] = share
    return pd.Series(vals, index=month_cols)


In [101]:
# Apply to all rows (or filter e.g., ACTIVE meters only)
# active_mask = cleaned['METER_STATUS'].eq('ACTIVE')
# cleaned.loc[active_mask, months_12] = cleaned.loc[active_mask].apply(backward_distribution_row, axis=1)
cleaned[months_12] = cleaned.apply(backward_distribution_row, axis=1)

In [102]:
# Recompute Total exactly from months
cleaned['Total'] = cleaned[months_12].sum(axis=1)

In [107]:
output_path = r"C:/Users/PC/Documents/Water datasets/2019/combined_backward_distributed_2019.xlsx"
cleaned.to_excel(output_path, index=False)